# Homework Assignment 11


## Instructions

Consider the reservoir shown below with the given properties that has been discretized into equal grid blocks.

![image](images/grid.png)

To be clear, there is a constant-rate injector of 1000 ft$^3$/day at $x$ = 5000 ft, $y$ = 5000 ft and a constant BHP well (producer) with $p_w$ = 800 psi at $x$ = 9000 ft, $y$ = 9000 ft. Both wells have a radius of 0.25 ft and no skin factor.

Use the code you wrote in [Assignment 9](https://github.com/PGE392K/assignment11) and add additional functionality to incorporate the wells.  The wells section of the inputs will look something like:

```yml
'wells':
    'rate':
        'locations': 
            - [0.0, 1.0]
            - [9999.0, 2.0]
        'values': [1000, 1000]
        'radii': [0.25, 0.25]
    'bhp':
        'locations': 
            - [6250.0, 1.0]
        'values': [800]
        'radii': [0.25]
        'skin factor': 0.0
```

notice that all the values are Python lists so that multiple wells of each type can be included.  The `'locations'` keyword has a value that is a list of lists.  Each tuple contains the $x,y$ Cartesian coordinate pair that gives the location of the well.  You must write some code that can take this $x,y$-pair and return the grid block number that the well resides in.  This should be general enough that changing the number of grids in the $x$ and $y$ directions still gives the correct grid block.  Once you know the grid block numbers for the wells, the changes to `fill_matrices()` should be relatively easy.

All of the old tests from the last few assignments are still in place, so your code must run in the absence of any well section in your inputs.

In [ ]:
import numpy as np
import scipy.sparse
import scipy.sparse.linalg
import matplotlib.pyplot as plt
import yaml

In [ ]:
class OneDimReservoir():
    
    def __init__(self, inputs):
        if isinstance(inputs, str):
            with open(inputs) as f:
                self.inputs = yaml.load(f, yaml.FullLoader)
        else:
            self.inputs = inputs
        
        self.Nx = self.inputs['numerical']['number of grids']['x']
        self.N = self.Nx
        self.L = self.inputs['reservoir']['length']
        
        # Delta_x as array
        if 'delta x' in self.inputs['numerical']:
            val = self.inputs['numerical']['delta x']
            self.delta_x = np.array(val) if isinstance(val, list) else np.full(self.N, val)
        else:
            self.delta_x = np.full(self.N, self.L / self.N)
        
        # Properties as arrays
        self.k = self._get_property('reservoir', 'permeability')
        self.phi = self._get_property('reservoir', 'porosity')
        self.A = self._get_property('reservoir', 'cross sectional area')
        
        self.delta_t = self.inputs['numerical']['time step']
        self.mu = self.inputs['fluid']['water']['viscosity']
        self.c = self.inputs['fluid']['water']['compressibility']
        
        self.apply_initial_conditions()
        self.fill_matrices()
        
        if 'plots' in self.inputs:
            self.p_plot = []
        
        return
    
    def _get_property(self, *path):
        val = self.inputs
        for key in path:
            val = val[key]
        if isinstance(val, (int, float)):
            return np.full(self.N, val)
        elif isinstance(val, list) and len(val) == self.N:
            return np.array(val)
        raise ValueError(f"Invalid property for {path}")
    
    def compute_transmissibility(self, i, j):
        if not (0 <= i < self.N and 0 <= j < self.N):
            raise ValueError("Invalid block indices")
        if i == j:
            return self.k[i] * self.A[i] / (self.mu * self.delta_x[i])
        elif abs(i - j) == 1:
            trans_i = self.compute_transmissibility(i, i)
            trans_j = self.compute_transmissibility(j, j)
            return 2 * trans_i * trans_j / (trans_i + trans_j)
        else:
            raise ValueError("Transmissibility only for same or adjacent blocks")
    
    def compute_accumulation(self, i):
        return self.phi[i] * self.c * self.A[i] * self.delta_x[i]
    
    def fill_matrices(self):
        conv = self.inputs['conversion factor']
        N = self.N
        acc_values = [self.compute_accumulation(i) for i in range(N)]
        self.B = scipy.sparse.diags([acc_values], [0], format='csr')
        
        self.Q = np.zeros(N)
        
        # Build T (with conv)
        lower = [conv * self.compute_transmissibility(i, i+1) for i in range(N-1)]
        upper = lower[:]  # Symmetric for this formulation
        main_diag = []
        for i in range(N):
            t_l = conv * self.compute_transmissibility(i, i-1) if i > 0 else 0.0
            t_r = conv * self.compute_transmissibility(i, i+1) if i < N-1 else 0.0
            main_diag.append(-(t_l + t_r))
        self.T = scipy.sparse.diags([lower, main_diag, upper], [-1, 0, 1], format='csr')
        
        # Apply boundary conditions
        self.T = self.T.tolil()
        left_bc = self.inputs['boundary conditions']['left']
        if left_bc['type'] == 'prescribed pressure':
            p_left = left_bc['value']
            t_left = conv * 2 * self.compute_transmissibility(0, 0)
            self.T[0, 0] -= t_left
            self.Q[0] += t_left * p_left
        elif left_bc['type'] == 'prescribed flux':
            self.Q[0] += left_bc['value']
        
        right_bc = self.inputs['boundary conditions']['right']
        if right_bc['type'] == 'prescribed pressure':
            p_right = right_bc['value']
            t_right = conv * 2 * self.compute_transmissibility(N-1, N-1)
            self.T[-1, -1] -= t_right
            self.Q[-1] += t_right * p_right
        elif right_bc['type'] == 'prescribed flux':
            self.Q[-1] += right_bc['value']
        
        self.T = self.T.tocsr()
        return
    
    def apply_initial_conditions(self):
        self.p = np.full(self.N, self.inputs['initial conditions']['pressure'])
        return
    
    def solve_one_step(self):
        solver_input = self.inputs['numerical']['solver']
        dt = self.delta_t
        p_old = self.p.copy()
        
        if isinstance(solver_input, str):
            if solver_input == 'explicit':
                theta = 1.0
            elif solver_input == 'implicit':
                theta = 0.0
            else:
                raise ValueError(f"Unknown solver: {solver_input}")
        elif isinstance(solver_input, dict) and 'mixed method' in solver_input:
            theta = solver_input['mixed method']['theta']
        else:
            raise ValueError("Invalid solver configuration")
        
        theta = 1 - theta  # Convention reversal
        
        acc_values = [self.compute_accumulation(i) for i in range(self.N)]
        B_dt_values = [acc / dt for acc in acc_values]
        B_dt = scipy.sparse.diags(B_dt_values, 0, format='csr')
        
        if theta == 0.0:
            inv_B_values = [dt / acc for acc in acc_values]
            inv_B = scipy.sparse.diags(inv_B_values, 0, format='csr')
            self.p = p_old + inv_B @ (self.T @ p_old + self.Q)
        else:
            A = B_dt - theta * self.T
            rhs = B_dt @ p_old + (1 - theta) * (self.T @ p_old) + self.Q
            self.p, _ = scipy.sparse.linalg.cg(A, rhs, atol=1.0e-8)
        
        return
    
    def solve(self):
        for i in range(self.inputs['numerical']['number of time steps']):
            self.solve_one_step()
            if 'plots' in self.inputs and i % self.inputs['plots']['frequency'] == 0:
                self.p_plot.append(self.get_solution().copy())
        return
    
    def plot(self):
        if self.p_plot:
            for pres in self.p_plot:
                plt.plot(pres)
        return
    
    def get_solution(self):
        return self.p

In [ ]:
import numpy as np
import scipy.sparse
import scipy.sparse.linalg
import matplotlib.pyplot as plt
import yaml

class TwoDimReservoir():
    
    def __init__(self, inputs):
        if isinstance(inputs, str):
            with open(inputs) as f:
                self.inputs = yaml.load(f, yaml.FullLoader)
        else:
            self.inputs = inputs
        
        self.Nx = self.inputs['numerical']['number of grids']['x']
        self.Ny = self.inputs['numerical']['number of grids']['y']
        self.N = self.Nx * self.Ny
        self.Lx = self.inputs['reservoir']['length']
        self.Ly = self.inputs['reservoir']['height']
        self.depth = self.inputs['reservoir']['depth']
        
        # Delta_x as array
        if 'delta x' in self.inputs['numerical']:
            val = self.inputs['numerical']['delta x']
            self.delta_x = np.array(val) if isinstance(val, list) else np.full(self.Nx, val)
        else:
            self.delta_x = np.full(self.Nx, self.Lx / self.Nx)
        
        # Delta_y as array
        if 'delta y' in self.inputs['numerical']:
            val = self.inputs['numerical']['delta y']
            self.delta_y = np.array(val) if isinstance(val, list) else np.full(self.Ny, val)
        else:
            self.delta_y = np.full(self.Ny, self.Ly / self.Ny)
        
        # Properties as arrays
        self.k = self._get_property('reservoir', 'permeability')
        self.phi = self._get_property('reservoir', 'porosity')
        
        self.delta_t = self.inputs['numerical']['time step']
        self.mu = self.inputs['fluid']['water']['viscosity']
        self.c = self.inputs['fluid']['water']['compressibility']
        
        self.apply_initial_conditions()
        self.fill_matrices()
        
        if 'plots' in self.inputs:
            self.p_plot = []
        
        return
    
    def _get_property(self, *path):
        val = self.inputs
        for key in path:
            val = val[key]
        if isinstance(val, (int, float)):
            return np.full(self.N, val)
        elif isinstance(val, list) and len(val) == self.N:
            return np.array(val)
        raise ValueError(f"Invalid property for {path}")
    
    def get_coords(self, idx):
        ix = idx % self.Nx
        iy = idx // self.Nx
        return ix, iy
    
    def get_block_number(self, x, y):
        x_edges = np.concatenate(([0], np.cumsum(self.delta_x)))
        ix = np.searchsorted(x_edges, x) - 1
        if x <= 0:
            ix = 0
        if ix < 0 or ix >= self.Nx:
            raise ValueError(f"x={x} out of bounds")
        
        y_edges = np.concatenate(([0], np.cumsum(self.delta_y)))
        iy = np.searchsorted(y_edges, y) - 1
        if y <= 0:
            iy = 0
        if iy < 0 or iy >= self.Ny:
            raise ValueError(f"y={y} out of bounds")
        
        return iy * self.Nx + ix
    
    def compute_transmissibility(self, i, j):
        if not (0 <= i < self.N and 0 <= j < self.N):
            raise ValueError("Invalid block indices")
        if i == j:
            ix, iy = self.get_coords(i)
            area = self.depth * self.delta_y[iy]
            return self.k[i] * area / (self.mu * self.delta_x[ix])
        
        ix_i, iy_i = self.get_coords(i)
        ix_j, iy_j = self.get_coords(j)
        k_i = self.k[i]
        k_j = self.k[j]
        
        if k_i == 0 or k_j == 0:
            return 0.0
        
        if iy_i == iy_j and abs(ix_i - ix_j) == 1:
            # x-direction
            area = self.depth * self.delta_y[iy_i]
            dx_i = self.delta_x[ix_i]
            dx_j = self.delta_x[ix_j]
        elif ix_i == ix_j and abs(iy_i - iy_j) == 1:
            # y-direction
            area = self.depth * self.delta_x[ix_i]
            dx_i = self.delta_y[iy_i]
            dx_j = self.delta_y[iy_j]
        else:
            raise ValueError("Blocks are not adjacent")
        
        return 2 * k_i * k_j * area / (self.mu * (k_j * dx_i + k_i * dx_j))
    
    def compute_accumulation(self, i):
        ix, iy = self.get_coords(i)
        return self.phi[i] * self.c * self.delta_x[ix] * self.delta_y[iy] * self.depth
    
    def fill_matrices(self):
        '''
           Assemble the transmisibility, accumulation matrices, and the flux
           vector.  Returns sparse data-structures
        '''
    
        conv = self.inputs['conversion factor']
        N = self.N
        acc_values = [self.compute_accumulation(i) for i in range(N)]
        self.B = scipy.sparse.diags(acc_values, format='csr')
        
        self.Q = np.zeros(N)
        
        self.T = scipy.sparse.lil_matrix((N, N))
        
        for idx in range(N):
            ix, iy = self.get_coords(idx)
            
            # left
            if ix > 0:
                j = idx - 1
                trans = conv * self.compute_transmissibility(idx, j)
                self.T[idx, j] = trans
                self.T[idx, idx] -= trans
            
            # right
            if ix < self.Nx - 1:
                j = idx + 1
                trans = conv * self.compute_transmissibility(idx, j)
                self.T[idx, j] = trans
                self.T[idx, idx] -= trans
            
            # bottom
            if iy > 0:
                j = idx - self.Nx
                trans = conv * self.compute_transmissibility(idx, j)
                self.T[idx, j] = trans
                self.T[idx, idx] -= trans
            
            # top
            if iy < self.Ny - 1:
                j = idx + self.Nx
                trans = conv * self.compute_transmissibility(idx, j)
                self.T[idx, j] = trans
                self.T[idx, idx] -= trans
        
        # Boundary conditions
        left_bc = self.inputs['boundary conditions']['left']
        total_area_x = self.depth * np.sum(self.delta_y)
        if left_bc['type'] == 'prescribed pressure':
            p_left = left_bc['value']
            for iy in range(self.Ny):
                i = iy * self.Nx
                area = self.depth * self.delta_y[iy]
                if self.k[i] == 0:
                    b_trans = 0.0
                else:
                    b_trans = conv * 2 * self.k[i] * area / (self.mu * self.delta_x[0])
                self.T[i, i] -= b_trans
                self.Q[i] += b_trans * p_left
        elif left_bc['type'] == 'prescribed flux':
            for iy in range(self.Ny):
                i = iy * self.Nx
                area = self.depth * self.delta_y[iy]
                q_block = left_bc['value'] * area / total_area_x
                self.Q[i] += q_block
        
        right_bc = self.inputs['boundary conditions']['right']
        if right_bc['type'] == 'prescribed pressure':
            p_right = right_bc['value']
            for iy in range(self.Ny):
                i = self.Nx - 1 + iy * self.Nx
                area = self.depth * self.delta_y[iy]
                if self.k[i] == 0:
                    b_trans = 0.0
                else:
                    b_trans = conv * 2 * self.k[i] * area / (self.mu * self.delta_x[self.Nx - 1])
                self.T[i, i] -= b_trans
                self.Q[i] += b_trans * p_right
        elif right_bc['type'] == 'prescribed flux':
            for iy in range(self.Ny):
                i = self.Nx - 1 + iy * self.Nx
                area = self.depth * self.delta_y[iy]
                q_block = right_bc['value'] * area / total_area_x
                self.Q[i] += q_block
        
        bottom_bc = self.inputs['boundary conditions']['bottom']
        total_area_y = self.depth * np.sum(self.delta_x)
        if bottom_bc['type'] == 'prescribed pressure':
            p_bottom = bottom_bc['value']
            for ix in range(self.Nx):
                i = ix
                area = self.depth * self.delta_x[ix]
                if self.k[i] == 0:
                    b_trans = 0.0
                else:
                    b_trans = conv * 2 * self.k[i] * area / (self.mu * self.delta_y[0])
                self.T[i, i] -= b_trans
                self.Q[i] += b_trans * p_bottom
        elif bottom_bc['type'] == 'prescribed flux':
            for ix in range(self.Nx):
                i = ix
                area = self.depth * self.delta_x[ix]
                q_block = bottom_bc['value'] * area / total_area_y
                self.Q[i] += q_block
        
        top_bc = self.inputs['boundary conditions']['top']
        if top_bc['type'] == 'prescribed pressure':
            p_top = top_bc['value']
            for ix in range(self.Nx):
                i = ix + (self.Ny - 1) * self.Nx
                area = self.depth * self.delta_x[ix]
                if self.k[i] == 0:
                    b_trans = 0.0
                else:
                    b_trans = conv * 2 * self.k[i] * area / (self.mu * self.delta_y[self.Ny - 1])
                self.T[i, i] -= b_trans
                self.Q[i] += b_trans * p_top
        elif top_bc['type'] == 'prescribed flux':
            for ix in range(self.Nx):
                i = ix + (self.Ny - 1) * self.Nx
                area = self.depth * self.delta_x[ix]
                q_block = top_bc['value'] * area / total_area_y
                self.Q[i] += q_block

        # Wells
        if 'wells' in self.inputs:
            wells = self.inputs['wells']
            if 'rate' in wells:
                rate_locs = wells['rate']['locations']
                rate_vals = wells['rate']['values']
                for loc, q in zip(rate_locs, rate_vals):
                    block = self.get_block_number(loc[0], loc[1])
                    self.Q[block] += q
            if 'bhp' in wells:
                bhp_locs = wells['bhp']['locations']
                bhp_vals = wells['bhp']['values']
                bhp_radii = wells['bhp']['radii']
                skin = wells['bhp'].get('skin factor', 0.0)
                for loc, pw, rw in zip(bhp_locs, bhp_vals, bhp_radii):
                    block = self.get_block_number(loc[0], loc[1])
                    ix, iy = self.get_coords(block)
                    dx = self.delta_x[ix]
                    dy = self.delta_y[iy]
                    kx = self.k[block]
                    ky = kx  # isotropic
                    req = 0.28 * np.sqrt(np.sqrt(ky / kx) * dx**2 + np.sqrt(kx / ky) * dy**2) / ( (ky / kx)**0.25 + (kx / ky)**0.25 )
                    ln_req_rw = np.log(req / rw) + skin
                    J = conv * 2 * np.pi * np.sqrt(kx * ky) * self.depth / (self.mu * ln_req_rw)
                    self.T[block, block] -= J
                    self.Q[block] += J * pw
        
        self.T = self.T.tocsr()
        return
    
    def apply_initial_conditions(self):
        self.p = np.full(self.N, self.inputs['initial conditions']['pressure'])
        return
    
    def solve_one_step(self):
        solver_input = self.inputs['numerical']['solver']
        dt = self.delta_t
        p_old = self.p.copy()
        
        if isinstance(solver_input, str):
            if solver_input == 'explicit':
                theta = 1.0
            elif solver_input == 'implicit':
                theta = 0.0
            else:
                raise ValueError(f"Unknown solver: {solver_input}")
        elif isinstance(solver_input, dict) and 'mixed method' in solver_input:
            theta = solver_input['mixed method']['theta']
        else:
            raise ValueError("Invalid solver input")
        
        lhs = self.B / dt - (1 - theta) * self.T
        rhs = (self.B / dt) @ p_old + theta * self.T @ p_old + self.Q
        self.p = scipy.sparse.linalg.spsolve(lhs, rhs)
        
        if 'plots' in self.inputs:
            self.p_plot.append(self.p.copy())
        
        return
    
    def solve(self):
        n_steps = self.inputs['numerical']['number of time steps']
        for _ in range(n_steps):
            self.solve_one_step()
        return
    
    def get_solution(self):
        return self.p